Hash 表

Hash 表主要包括两个基本操作
* 计算Hash函数的值
* 定位到对应链表中依次遍历、比较

为了解决Hash映射后的冲突问题，一般使用“开散列”的解决方案。建立一个邻接表结构，以Hash函数的值域作为表头数组head，映射后的值相同的原始信息被分到同一类，构成一个链表接在对应的表头之后，链表的节点上可以保存原始信息和一些统计数据。

1. 例题：雪花雪花雪花 [Acwing137](https://www.acwing.com/problem/content/139/)

* 题解
  * 使用hash, **但是我完全不知道为啥上来就想到Hash了！！！**书中是下面这样说的：
    * **显然，对于两片形状相同的雪花，他们六个角的长度之和、长度之积都相等，因此它们的Hash函数值也相等。**
  * 建立一个Hash表，把N片雪花依次插入。对于每片雪花$a_{i,1},a_{i,2},...,a_{i,6}$，我们直接扫描表头 H($a_{i,1},a_{i,2},...,a_{i,6}$)对应的链表，检查是否存在与$a_{i,1},a_{i,2},...,a_{i,6}$形状相同的雪花即可。

In [ ]:
import collections

MOD = 99991
Snow = collections.namedtuple('Snow', 'arms next')
snows = [None] * 100000

def check(arms, idx):
    """遍历所有雪花"""
    p = snows[idx]
    while p is not None:            # 有些值相同可能被映射到相同的链中，这里所以遍历一下
        for i in range(6):
            # 寻找对比起点
            if arms[i] == p.arms[0]:
                j = i
                # 正向
                for k in range(1,6):
                    j = (j + 1) % 6
                    if arms[j] != p.arms[k]:
                        break
                else:
                    return True
                j = i
                # 反向
                for k in range(5, 0, -1):
                    j = (j + 1) % 6
                    if arms[j] != p.arms[k]:
                        break
                else:
                    return True
        p = p.next
    """将现在的雪花插入"""
    node = Snow(arms, snows[idx])
    snows[idx] = node
    return False

def main():
    n = int(input())
    for i in range(n):
        temp = list(map(int, input().split()))
        idx = sum(temp)%MOD
        if check(temp, idx):
            print('Twin snowflakes found.')
            return
    print('No two snowflakes are alike.')


if __name__ == '__main__':
    main()

2. 字符串 Hash  **感觉挺重要的**

把一个字符串映射成一个整数，这样查找字符串时间复杂度就会变得简单

* 操作
  * 把整个字符串看成一个P进制数，然后对q取模，得到的值就是hash值
  * 给所有可能出现的字符都赋值，比如a~z赋值为 1~26，假设这个映射函数为H
  * 对一个长度为n的字符串S，计算它的整数表示为 $val = \sum_{i=0}^{n} H[S[i]]*P^{n-i}$
  * 那么它的hash值就是 val%q
  * **一般p取131或者13331q=$2^{64}$**，q取$2^{64}$是因为我们可以直接用unsigned long long，溢出也不去管它，那么得到的val就相当于取模后的 val%q，这样可以少了取模这一步。
  * 使用上面的p和q，正常做题可以默认是没有冲突的。

* 性质
  * 假设上述将字符串映射到hash值的过程看做一个函数$f(i)=hash_i$：表示S前i个字符组成的前缀的hash值
  * $f(i+1) = f(i)*p + H[S[i+1]]$
  * $f(i-j) = f(i) - f(j-1)*p^{i-j+1}, j <i$ ,后面*$p^{i-j+1}$是因为我们表示的时候，默认字符在左边为高位。

2.1 兔子兔子 [Acwing138]((https://www.acwing.com/problem/content/140/)

这里先用c++的代码实现，因为python中没有unsigned long long这个限制

In [ ]:
#include <iostream>
#include <string.h>

using namespace std;
typedef unsigned long long ULL;
const int N = 100010,base = 131;

char str[N];
ULL h[N],p[N];  // p用来存进位

ULL get(int l,int r){
    return h[r] - h[l-1]*p[r-l+1];
}

int main(){
    scanf("%s",str+1);
    int n = strlen(str+1);
    // 这就是字符串hash的实现方法
    for(int i=1;i <= n; i++){
        h[i] = h[i-1] * base + str[i]-'a' + 1;
        p[i] = p[i-1] * base;
    }
    int m;
    cin >> m;
    while (m--){
        int l1,r1,l2,r2;
        scanf("%d%d%d%d",&l1,&r1,&l2,&r2);
        if (get(l1,r1) == get(l2,r2)) puts("Yes");
        else puts("No");
    }
    return 0;
}


2.2 最长回文字串 [Acwing139](https://www.acwing.com/problem/content/141/)

* 技巧
  * 在每个字符串中插入一个#，这样的话，无论奇回文还是偶回文，都可以编程奇回文

* 题解
  1. 使用暴力
  2. 遍历每个字符，分奇偶，同时向两段扩展，O($n^2$)
  3. 对第2种方法的改进，使用字符串hash+二分查找进行判断，O(nlogn)
  4. Manacher算法

* blog
  * [最长回文字串](https://www.jianshu.com/p/799bc53d4e3d)

3. 后缀数组 [Acwing140](https://www.acwing.com/problem/content/142/)

比较难，跳过了